In [1]:
#import

import numpy as np
import matplotlib.pyplot as plt
import os
import rasterio
from pathlib import Path                                    #Alternative zum OS für Pfade
from pprint import pprint as pp
from sklearn.ensemble import RandomForestClassifier as RF   #Random Forest Klassifikator
from sklearn.model_selection import train_test_split        #Für Einteilung des Datensatzes in Trainings- und Validierungsdatensatzes
from sklearn.metrics import plot_confusion_matrix           #


In [2]:
# Pfade für die Umgebung Setzen!

dirname = "D:\_Programmieren\Landcover"
bands_0_2m = []
path_rgb_img = Path(os.path.join(dirname, 'RGB\\2018_t'))
path_cir_img = Path(os.path.join(dirname, 'CIR_all\\2018_t'))
path_ortho = Path(os.path.join(dirname, 'RGB\\2018_t\\2121-34_201807.tif'))
path_cir = Path(os.path.join(dirname, 'CIR_all\\2018_t\\2121-34_201807.tif'))

path_mask_poly = os.path.join(dirname, 'MaskData\\mask_poly.shp')
path_raster_mask = os.path.join(dirname, 'MaskData/maske_raster_g.tif')


In [3]:
# # rasterize training and test data (nur beim ersten mal notwendig)

# xmin, ymin, xmax, ymax = 51250, 205000, 60000, 214000
# res = 0.2
# datatype_out = "Int16"
# burn_field = "Kategorie"
# nodata = "-1"

# training_vector = path_mask_poly
# training_raster = path_raster_mask

# cmd = f"gdal_rasterize -a {burn_field} -of GTiff -te {xmin} {ymin} {xmax} {ymax} -tr {res} {res} -a_nodata {nodata} -ot {datatype_out} {training_vector} {training_raster}"
# print(cmd)

# os.system(cmd)



gdal_rasterize -a Kategorie -of GTiff -te 51250 205000 60000 214000 -tr 0.2 0.2 -a_nodata -1 -ot Int16 D:\_Programmieren\Landcover\MaskData\mask_poly.shp D:\_Programmieren\Landcover\MaskData/maske_raster_g.tif


0

In [3]:
############## Hilfsfunktion zum Rastereinlesen ####################

def read_img(path_to_img):
    with rasterio.open(path_to_img, "r") as img:
        return img.read(1).astype(np.float32)

def ortho_to_3bands(path_to_img):
    with rasterio.open(path_to_img, "r") as img:
        return img.read(1).astype(np.float32),img.read(2).astype(np.float32),img.read(3).astype(np.float32)
 

In [4]:
##################### Bänder Einlesen #####################
bands = []
for f in path_rgb_img.glob("*tif"):
    #print(f) #sollte hier alle benötigten Bänder ausdrücke werden
    bands.append(ortho_to_3bands(f)[0])
    bands.append(ortho_to_3bands(f)[1])
    bands.append(ortho_to_3bands(f)[2])

# for f in path_cir_img.glob("*tif"):
#     #print(f) #sollte hier alle benötigten Bänder ausdrücke werden
#     bands.append(ortho_to_3bands(f)[0])
#     bands.append(ortho_to_3bands(f)[1])
#     bands.append(ortho_to_3bands(f)[2])
#print(bands)
#Stapelt Bänder
bands = np.dstack(bands)
print(bands.shape) #  Gibt Reihen, Spalten, Kanäle aus.

(45000, 43750, 3)


In [5]:
# read training/test data
# just for reference
mapping = {
    0: "Wiese",
    1: "Wald",
    2: "Fels",
    3: "Schnee und Eis",
    4: "Straße",
    5: "Wasser"
}
labels = read_img(path_raster_mask)

print(labels.shape)
print(np.unique(labels, return_counts=True))


# reshape training test data 
labels_1d = labels[labels>=0].reshape((-1))
print(labels_1d.shape)


# split into 
X_train, X_test, y_train, y_test = train_test_split(bands[labels>=0,:], labels_1d, test_size=0.3, random_state=0, shuffle=True)



(45000, 43750)
(array([-1.,  0.,  1.,  2.,  3.,  4.,  5.], dtype=float32), array([1937371357,    2467200,    8815037,    5821140,   14164975,
            19048,      91243], dtype=int64))
(31378643,)


In [7]:
# instantiate classificator
estimatorList = [10] # Hier können mehr werte eingegeben werden um zu vergleichen 
for estimator in estimatorList:
    
    rf = RF(n_estimators=estimator, n_jobs=-1, oob_score=True)

    # train our model
    print("training model...")
    rf.fit(X_train, y_train)
    print("model trained ... ")
    #############################################################################
    ################## Versuch auf fremden Daten anzuwenden #####################
    #############################################################################
    test_rgb_img = Path(os.path.join(dirname, 'RGB\\2018'))
    test_cir_img = Path(os.path.join(dirname, 'CIR_all\\2018_v'))

    for fObj in test_rgb_img.glob("*tif"): #jede datei in besagtem ordner
        print("#######################",fObj)
        print(Path(fObj).stem)
        rgbP = str(test_rgb_img)+"\\"+str(Path(fObj).stem)+".tif"
        cirP = str(test_cir_img)+"\\"+str(Path(fObj).stem)+".tif"
        bandsTest = []
        bandsTest.append(ortho_to_3bands(rgbP)[0])
        bandsTest.append(ortho_to_3bands(rgbP)[1])
        bandsTest.append(ortho_to_3bands(rgbP)[2])

        #print(f) #sollte hier alle benötigten Bänder ausdrücke werden
        # bandsTest.append(ortho_to_3bands(cirP)[0])
        # bandsTest.append(ortho_to_3bands(cirP)[1])
        # bandsTest.append(ortho_to_3bands(cirP)[2])
        bandsTest = np.dstack(bandsTest)
        print(bandsTest.shape)
        #############################################################################
        #############################################################################
        #############################################################################

        # extract information for reshaping
        rows, cols, n_bands = bandsTest.shape
        


        # transform input data into 2d array of shape 
        X_predict = bandsTest.reshape((rows*cols, n_bands))


        # apply our model on (unseen) data
        y_pred = rf.predict(X_predict)

        # reshape to shape of (rows, cols) for output as 2d gis raster
        y_pred_2d = y_pred.reshape((rows, cols))


        template = {}
        with rasterio.open(fObj, "r") as img:
            template["transform"] = img.transform
            template["crs"] = img.crs
            template["height"] = img.height
            template["width"] = img.width

        with rasterio.open(
            dirname+"\\results\\testMultiple\\prediction_nE_"+str(estimator)+str(Path(fObj).stem)+".tif",
            "w",
            driver="GTiff",
            height=y_pred_2d.shape[0],
            width=y_pred_2d.shape[1],
            count=1,
            dtype=y_pred_2d.dtype,
            crs=template["crs"],
            transform=template["transform"]) as fobj:
            fobj.write(y_pred_2d, 1)
        print("Estimators: "+str(estimator)+" done.")
print("Finished.")

training model...


C:\Users\ronny\.conda\envs\rs\lib\site-packages\sklearn\ensemble\_forest.py:541: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
C:\Users\ronny\.conda\envs\rs\lib\site-packages\sklearn\ensemble\_forest.py:546: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


model trained ... 
####################### D:\_Programmieren\Landcover\RGB\2018\2121-02_201807.tif
2121-02_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-03_201807.tif
2121-03_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-04_201807.tif
2121-04_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-05_201807.tif
2121-05_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-06_201807.tif
2121-06_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-07_201807.tif
2121-07_201807
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2018\2121-08_201909.tif
2121-08_201909
(5000, 6250, 3)
Estimators: 10 done.
####################### D:\_Programmieren\Landcover\RGB\2

KeyboardInterrupt: 